# Project Pagu - Cat Detector


## Como rodar este notebook (passo-a-passo)

1. Abra o Google Colab e monte o Google Drive (se necessário):
```python
from google.colab import drive
drive.mount('/content/drive')
```
2. Edite a célula de configuração no início do notebook (input_path, output_path, etc.).
3. Execute as células na ordem (Config -> Imports -> Funções -> Executar).

Se quiser usar GPU: vá em *Runtime > Change runtime type* e selecione GPU. Deixe `use_gpu = True` na célula de configuração; caso contrário deixe `False`.


In [ ]:
# ======= CÉLULA DE CONFIGURAÇÃO (edite somente esta célula) =======
# Instruções (leia antes de executar):
# 1° defina o caminho de entrada (input_path): diretório onde está o(s) vídeo(s) a serem processados.
# 2° defina o caminho de saída (output_path): diretório onde os resultados serão salvos.
# 3° GPU opcional: se estiver em Colab e quiser usar GPU, deixe use_gpu = True; caso contrário, False.
# 4° Ajuste chunk_duration para duração (em segundos) de cada pedaço de vídeo (se aplicável).
# 5° Não altere o restante do notebook, a menos que saiba o que faz.
#
# Exemplos:
# input_path = '/content/drive/MyDrive/meus_videos'
# output_path = '/content/drive/MyDrive/colab_outputs/cats_chunks'
# use_gpu = True
# chunk_duration = 8
#
# Depois de editar, execute as células na ordem: primeiro esta (config), depois 'Imports', depois 'Funções', e por fim 'Executar'.


# ===== Valores padrão (substitua conforme necessário) =====
input_path = '/content/drive/MyDrive/colab_inputs'
output_path = '/content/drive/MyDrive/colab_outputs/cats_chunks'
use_gpu = True  # tente usar GPU se disponível; o código detecta e faz fallback para CPU
chunk_duration = 8 # aumente esse valor caso o vídeo for de 10 min ou mais
concat_outputs = True  # concatenar chunks ao final


In [ ]:
# ===== Imports organizados =====
# Comentários em Português explicando cada import
import os
import math
from pathlib import Path
from typing import List, Tuple
# moviepy para processamento de vídeo (funciona em CPU e GPU para leitura/export se ffmpeg usar aceleração)
from moviepy.editor import VideoFileClip, concatenate_videoclips
# Trata exceções e logs simples para usuário
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


In [ ]:
# ===== Funções utilitárias =====
# Estas funções são genéricas: criação de pastas, listagem de vídeos e verificação de extensões.
def ensure_dir(path: str):
    """Cria diretório se não existir."""
    Path(path).mkdir(parents=True, exist_ok=True)

def list_video_files(folder: str) -> List[str]:
    """Retorna lista de arquivos de vídeo comuns no diretório informado."""
    exts = ('.mp4', '.mov', '.avi', '.mkv')
    files = []
    folder_p = Path(folder)
    if not folder_p.exists():
        logging.warning(f"Pasta de entrada não existe: {folder}")
        return files
    for entry in sorted(folder_p.iterdir()):
        if entry.is_file() and entry.suffix.lower() in exts:
            files.append(str(entry))
    return files

def safe_filename(path: str) -> str:
    return Path(path).stem


In [ ]:
# ===== Funções principais (mantêm a lógica original, refatoradas) =====
def process_video_in_chunks(input_folder, output_folder, chunk_duration=8):
    """... existing docstring ..."""
    import os
    import math
    from moviepy.editor import VideoFileClip

    os.makedirs(output_folder, exist_ok=True)
    allowed_ext = ('.mp4', '.mov', '.avi', '.mkv', '.webm', '.flv', '.mpeg', '.mpg', '.m4v')
    processed_files = []

    for filename in sorted(os.listdir(input_folder)):
        input_path = os.path.join(input_folder, filename)

        if os.path.isdir(input_path) or filename.startswith('.') or filename.lower().endswith('.txt'):
            continue
        if not filename.lower().endswith(allowed_ext):
            print(f"Skipping non-video file: {input_path}")
            continue

        print(f"Processing: {input_path}")
        try:
            clip = VideoFileClip(input_path)
        except Exception as e:
            print(f"Failed to open {input_path}: {e}")
            continue

        try:
            duration = clip.duration
            n_chunks = max(1, math.ceil(duration / float(chunk_duration)))
            for i in range(n_chunks):
                start = i * chunk_duration
                end = min(duration, (i + 1) * chunk_duration)
                subclip = clip.subclip(start, end)
                base_name = os.path.splitext(filename)[0]
                out_name = f"{base_name}_chunk_{i+1}.mp4"
                out_path = os.path.join(output_folder, out_name)

                try:
                    subclip.write_videofile(out_path, codec='libx264', audio=False)
                    processed_files.append(out_path)
                except Exception as e:
                    print(f"Failed to write chunk {i+1} for {filename}: {e}")
                finally:
                    try: subclip.reader.close()
                    except Exception: pass
                    if getattr(subclip, 'audio', None):
                        try: subclip.audio.reader.close_proc()
                        except Exception: pass
        finally:
            try: clip.reader.close()
            except Exception: pass
            if getattr(clip, 'audio', None):
                try: clip.audio.reader.close_proc()
                except Exception: pass

    return processed_files


# --- CELL BOUNDARY ---

process_video_in_chunks(
    "/content/drive/MyDrive/colab_inputs/cats/",
    "/content/drive/MyDrive/colab_outputs/cats_chunks/",
    chunk_duration=8
)


# --- CELL BOUNDARY ---


from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
from ultralytics import YOLO
import cv2
import numpy as np
from tqdm import tqdm

# === Paths ===
OUTPUT_DRIVE_FOLDER = '/content/drive/MyDrive/colab_outputs/cats_chunks'
STYLIZED_FOLDER = '/content/drive/MyDrive/colab_outputs/cats_stylized'
os.makedirs(STYLIZED_FOLDER, exist_ok=True)

# List of processed video files
processed_files = sorted([
    os.path.join(OUTPUT_DRIVE_FOLDER, f)
    for f in os.listdir(OUTPUT_DRIVE_FOLDER)
    if f.lower().endswith(".mp4")
])

if not processed_files:
    print("Nenhum arquivo .mp4 encontrado em", OUTPUT_DRIVE_FOLDER)
else:
    print(f"{len(processed_files)} arquivos de vídeo encontrados para processamento.")

final_output_path = os.path.join(OUTPUT_DRIVE_FOLDER, 'ALL_PROCESSED_concat.mp4')

# === Load YOLO model once (GPU if available) ===
model = YOLO("yolov8s.pt")
#model.to('cuda')
CAT_CLASS_ID = 15  # COCO id for cat

# === Function to process each chunk with YOLO + background blur ===
def stylize_chunk(input_path, output_path, model, blur_sigma=25.0):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Could not open {input_path}")
        return False

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    track_results = model.track(source=input_path, classes=[CAT_CLASS_ID], persist=True, show=False, verbose=False)

    for res in tqdm(track_results, total=total, desc=f"Processing {os.path.basename(input_path)}"):
        frame = res.orig_img.copy()
        h, w = frame.shape[:2]
        mask = np.zeros((h, w), dtype=np.uint8)

        if hasattr(res, "boxes") and res.boxes is not None:
            try:
                boxes = res.boxes.xyxy.cpu().numpy()
            except Exception:
                boxes = np.array(res.boxes.xyxy)
            for box in boxes:
                x1, y1, x2, y2 = [int(v) for v in box]
                cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)

        ksize = int(max(3, (int(blur_sigma) * 4 // 2) * 2 + 1))
        blurred = cv2.GaussianBlur(frame, (ksize, ksize), blur_sigma)
        mask_3c = np.repeat(mask[:, :, None].astype(bool), 3, axis=2)
        output_frame = np.where(mask_3c, frame, blurred)

        # Thin white boxes
        for box in boxes if len(mask.nonzero()[0]) else []:
            x1, y1, x2, y2 = [int(v) for v in box]
            cv2.rectangle(output_frame, (x1, y1), (x2, y2), (255, 255, 255), 1, cv2.LINE_AA)

        out.write(output_frame)

    out.release()
    cap.release()
    print(f"Saved stylized chunk: {output_path}")
    return True


# === Apply processing to each chunk ===
stylized_files = []
for chunk in processed_files:
    if os.path.exists(chunk):
        out_path = os.path.join(STYLIZED_FOLDER, os.path.basename(chunk).replace(".mp4", "_processed.mp4"))
        if stylize_chunk(chunk, out_path, model):
            stylized_files.append(out_path)

# === Concatenate stylized chunks into final video ===
if stylized_files:
    list_file = os.path.join(STYLIZED_FOLDER, "clips.txt")
    with open(list_file, "w") as f:
        for file_path in stylized_files:
            f.write(f"file '{file_path}'\n")

    final_output_path = os.path.join(STYLIZED_FOLDER, 'ALL_PROCESSED_concat.mp4')
    print("Concatenating all processed clips...")
    !ffmpeg -y -f concat -safe 0 -i "{list_file}" -c:v h264_nvenc -preset fast -c:a aac "{final_output_path}"
    print(f"Final video saved: {final_output_path}")
else:
    print("No processed chunks found.")



In [ ]:

import os

# Folder where your processed videos are stored
OUTPUT_DRIVE_FOLDER = '/content/drive/MyDrive/colab_outputs/cats_stylized'

# Create a text file listing all processed clips
list_file = os.path.join(OUTPUT_DRIVE_FOLDER, "clips.txt")
final_output_path = os.path.join(OUTPUT_DRIVE_FOLDER, "ALL_PROCESSED_concat.mp4")

with open(list_file, "w") as f:
    for filename in sorted(os.listdir(OUTPUT_DRIVE_FOLDER)):
        if filename.endswith(".mp4") and "processed" in filename:
            f.write(f"file '{os.path.join(OUTPUT_DRIVE_FOLDER, filename)}'\n")

# Run ffmpeg (CPU-safe fallback)
!ffmpeg -y -f concat -safe 0 -i "{list_file}" -c:v libx264 -preset fast -crf 18 -c:a aac "{final_output_path}"


In [ ]:
def concatenate_chunks(input_files: List[str], output_path: str) -> str:
    """Concatena uma lista de arquivos de vídeo (assume same resolution/fps) em um único arquivo."""
    if not input_files:
        raise ValueError('Nenhum arquivo para concatenar.')
    clips = []
    for f in input_files:
        try:
            clips.append(VideoFileClip(f))
        except Exception as e:
            logging.warning(f'Falha ao abrir {f} para concatenação: {e}')
    if not clips:
        raise ValueError('Nenhum clipe válido para concatenar.')
    final = concatenate_videoclips(clips, method='compose')
    ensure_dir(os.path.dirname(output_path))
    final.write_videofile(output_path, verbose=False, logger=None)
    for c in clips:
        c.close()
    final.close()
    return output_path


In [ ]:
# ===== Célula de execução (edite apenas a célula de configuração acima) =====
if __name__ == '__main__':
    logging.info('Iniciando processamento com as configurações fornecidas...')
    ensure_dir(output_path)
    generated = process_video_in_chunks(input_path, output_path, chunk_duration=chunk_duration)
    logging.info(f'Arquivos gerados: {len(generated)}')
    if concat_outputs and generated:
        # cria um arquivo final concatenado chamado final_output.mp4 dentro do output_path
        final_path = os.path.join(output_path, 'final_output.mp4')
        try:
            concatenate_chunks(generated, final_path)
            logging.info(f'Arquivo concatenado salvo em: {final_path}')
        except Exception as e:
            logging.error(f'Falha na concatenação: {e}')
    logging.info('Processamento finalizado. Verifique a pasta de saída para os resultados.')
